# Identifying Ethnicity in OpenSAFELY-TPP
This short report describes how ethnicity can be identified in the OpenSAFELY-TPP database, and the strengths and weaknesses of the methods. This is a living document that will be updated to reflect changes to the OpenSAFELY-TPP database and the patient records within.

## OpenSAFELY
OpenSAFELY is an analytics platform for conducting analyses on Electronic Health Records inside the secure environment where the records are held. This has multiple benefits: 

* We don't transport large volumes of potentially disclosive pseudonymised patient data outside of the secure environments for analysis
* Analyses can run in near real-time as records are ready for analysis as soon as they appear in the secure environment
* All infrastructure and analysis code is stored in GitHub repositories, which are open for security review, scientific review, and re-use

A key feature of OpenSAFELY is the use of study definitions, which are formal specifications of the datasets to be generated from the OpenSAFELY database. This takes care of much of the complex EHR data wrangling required to create a dataset in an analysis-ready format. It also creates a library of standardised and validated variable definitions that can be deployed consistently across multiple projects. 

The purpose of this report is to describe all such variables that relate to BMI, their relative strengths and weaknesses, in what scenarios they are best deployed. It will also describe potential future definitions that have not yet been implemented.

## Available Records
OpenSAFELY-TPP runs inside TPP’s data centre which contains the primary care records for all patients registered at practices using TPP’s SystmOne Clinical Information System. This data centre also imports external datasets from other sources, including A&E attendances and hospital admissions from NHS Digital’s Secondary Use Service, and death registrations from the ONS. More information on available data sources can be found within the OpenSAFELY documentation. 

In [1]:
from IPython.display import display, Markdown
from lib import *

pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None 

In [2]:
### CONFIGURE OPTIONS HERE ###

# Import file
input_path = '../output/data/input.feather'

# Definitions
definitions = ['ethnicity_snomed_5', 'ethnicity_5']

other_vars = []

# Dates
date_min = '2019-01-01'
date_max = '2019-12-31'
time_delta = 'M'

# Null value – 0 or NA
null = "0"

# Covariates
demographic_covariates = ['age_band', 'sex', 'region', 'imd']
clinical_covariates = ['dementia', 'diabetes', 'hypertension', 'learning_disability']

In [3]:
df_clean = import_clean(input_path, definitions, other_vars, demographic_covariates, clinical_covariates, null, time_delta, dates=False)

## Results

### Count of Patients

In [4]:
patient_counts(df_clean, definitions, demographic_covariates, clinical_covariates)

ethnicity_snomed_5_filled  \
group               subgroup                                             
population          N                                             1000   
                    with records                                   680   
age_band            0-19                                            95   
                    20-29                                           90   
                    30-39                                           80   
                    40-49                                           75   
                    50-59                                           80   
                    60-69                                           90   
                    70-79                                           95   
                    80+                                             75   
sex                 F                                              330   
                    M                                              350   
region              East Midlands                                   40   
                    East of England                                 40   
                    London                                          85   
                    North East                                      45   
                    North West                                      45   
                    South East                                      95   
                    West Midlands                                   45   
                    Yorkshire and the Humber                        40   
imd                 100                                            140   
                    200                                            145   
                    300                                            120   
                    400                                            150   
                    500                                            125   
dementia            False                                          675   
                    True                                             -   
diabetes            False                                          670   
                    True                                            10   
hypertension        False                                          680   
                    True                                             -   
learning_disability False                                          670   
                    True                                            10   

                                             ethnicity_snomed_5_pct  \
group               subgroup                                          
population          N                                           100   
                    with records                                 68   
age_band            0-19                                        9.5   
                    20-29                                         9   
                    30-39                                         8   
                    40-49                                       7.5   
                    50-59                                         8   
                    60-69                                         9   
                    70-79                                       9.5   
                    80+                                         7.5   
sex                 F                                            33   
                    M                                            35   
region              East Midlands                                 4   
                    East of England                               4   
                    London                                      8.5   
                    North East                                  4.5   
                    North West                                  4.5   
                    South East                                  9.5   
                    West Midlands 

### Count of Missings

In [5]:
patient_counts(df_clean, definitions, demographic_covariates, clinical_covariates, missing=True)

ethnicity_snomed_5_missing  \
group               subgroup                                              
population          N                                              1000   
                    missing records                                 320   
age_band            0-19                                             35   
                    20-29                                            40   
                    30-39                                            40   
                    40-49                                            45   
                    50-59                                            40   
                    60-69                                            40   
                    70-79                                            40   
                    80+                                              40   
sex                 F                                               160   
                    M                                               160   
region              East Midlands                                    25   
                    East of England                                  20   
                    London                                           45   
                    North East                                       20   
                    North West                                       20   
                    South East                                       40   
                    West Midlands                                    25   
                    Yorkshire and the Humber                         20   
imd                 100                                              70   
                    200                                              50   
                    300                                              55   
                    400                                              70   
                    500                                              70   
dementia            False                                           315   
                    True                                              -   
diabetes            False                                           320   
                    True                                              -   
hypertension        False                                           310   
                    True                                             10   
learning_disability False                                           320   
                    True                                              -   

                                             ethnicity_snomed_5_pct  \
group               subgroup                                          
population          N                                           100   
                    missing records                              32   
age_band            0-19                                        3.5   
                    20-29                                         4   
                    30-39                                         4   
                    40-49                                       4.5   
                    50-59                                         4   
                    60-69                                         4   
                    70-79                                         4   
                    80+                                           4   
sex                 F                                            16   
                    M                                            16   
region              East Midlands                               2.5   
                    East of England                               2   
                    London                                      4.5   
                    North East                                    2   
                    North West                                    2   
                    South East                                    4   

### Overlapping Definitions

In [11]:
print(df_clean)
display(df_clean['ethnicity_snomed_5','white_count'].groupby(definitions[0]))

display_heatmap(df_clean, definitions)

     patient_id ethnicity_snomed_5 ethnicity_5 age_band sex  \
0            24                  2         NaN     0-19   M   
1            44                  3           1    30-39   F   
2            59                NaN           4    60-69   F   
3            86                  2         NaN    20-29   M   
4            88                  4           3    30-39   M   
..          ...                ...         ...      ...  ..   
995        9937                  4           5    60-69   F   
996        9952                  1           1    50-59   F   
997        9964                  5           4    20-29   M   
998        9986                  2         NaN    50-59   M   
999        9995                  2           3    70-79   M   

                       region  imd  dementia  diabetes  hypertension  \
0                         NaN  100     False     False         False   
1                  North East  200     False     False         False   
2                      Lond

KeyError: ('ethnicity_snomed_5', 'white_count')